In [ ]:
#!/usr/bin/env python
# coding: utf-8

'''
import SQL
'''

# In[ ]:
# ** import package **
import os
import sys
import json
import pathlib
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import traceback
from tqdm import tqdm
from datetime import timedelta
from _utils.customlogger import customlogger as CL

pd.set_option('display.max_colwidth', -1)  #각 컬럼 width 최대로 
pd.set_option('display.max_rows', 50)      # display 50개 까지 

In [ ]:
# In[ ]:
# ** loading config **
with open('./../{}'.format("config.json")) as file:
    cfg = json.load(file)

In [ ]:
# In[ ]:
# ** loading info **
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
current_date = cfg["working_date"]
curr_file_name = os.path.splitext(os.path.basename(os.path.abspath('')))[0]

In [ ]:
# In[ ]:
# **create Logger**
log = CL("custom_logger")
pathlib.Path.mkdir(pathlib.Path('{}/_log/'.format(parent_dir)), mode=0o777, parents=True, exist_ok=True)
log = log.create_logger(file_name="../_log/{}.log".format(curr_file_name), mode="a", level="DEBUG")  
log.debug('start {}'.format(curr_file_name))

In [ ]:
# In[ ]:
# ** connection DataBase **
if (cfg["dbms"]=="postgresql"):
    db_cfg = cfg["postgresql"]
    import psycopg2 as pg
    conn = pg.connect(host=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], dbname=db_cfg['@database']) 
    log.debug("postgresql connect")
    
elif (cfg["dbms"]=="mssql"):
    db_cfg = cfg["mssql"]
    import pymssql
    conn= pymssql.connect(server=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], database=db_cfg['@database'], as_dict=False)
    log.debug("mssql connect")
    
elif (cfg["dbms"]=="mysql"):
    db_cfg = cfg["mysql"]
    import pymysql
    conn= pymysql.connect(host=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], db=db_cfg['@database'], charset='utf8', autocommit=True)
    log.debug("mysql connect")
    
else:
    log.warning("set config.json - sql - dbms : mssql or postgresql")

In [ ]:
apachepsvar_df = pd.read_sql("SELECT * FROM eicu.APACHEAPSVAR", con=conn)

In [ ]:
patient_df = pd.read_sql("SELECT * FROM eicu.PATIENT", con=conn)

In [ ]:
microlab_df = pd.read_sql("SELECT * FROM eicu.MICROLAB", con=conn)

In [10]:
lab_df = pd.read_sql("SELECT * FROM eicu.LAB", con=conn)

In [11]:
medication_df = pd.read_sql("SELECT * FROM eicu.MEDICATION", con=conn)

In [17]:
admissiondrug_df = pd.read_sql("SELECT * FROM eicu.ADMISSIONDRUG", con=conn)

In [15]:
apachepsvar_df.to_feather('apachepsvar_df.feather')
patient_df.to_feather('patient_df.feather')
microlab_df.to_feather('microlab_df.feather')
lab_df.to_feather('lab_df.feather')
medication_df.to_feather('medication_df.feather')
admissiondrug_df.to_feather('admissiondrug_df.feather')

In [ ]:
apachepsvar_df = pd.read_feather('apachepsvar_df.feather')
patient_df = pd.read_feather('patient_df.feather')
microlab_df = pd.read_feather('microlab_df.feather')
lab_df = pd.read_feather('lab_df.feather')
medication_df = pd.read_feather('medication_df.feather')
admissiondrug_df = pd.read_feather('admissiondrug_df.feather')

In [21]:
patient_apc_df = pd.merge(patient_df, apachepsvar_df, left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_med_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], medication_df[['patientunitstayid', 'drugname']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_lab_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], lab_df[['patientunitstayid', 'labname', 'labresult']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_adm_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], admissiondrug_df[['patientunitstayid', 'drugname']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')

In [35]:
patient_med_df.rename(columns={'drugname':'concept_name'}, inplace=True)
patient_med_df['concept_value'] = 1
patient_med_df['concept_table'] = 'med'
patient_lab_df.rename(columns={'labname':'concept_name','labresult':'concept_value'}, inplace=True)
patient_lab_df['concept_table'] = 'lab'
patient_adm_df.rename(columns={'drugname':'concept_name'}, inplace=True)
patient_adm_df['concept_value'] = 1
patient_adm_df['concept_table'] = 'adm'

In [36]:
concat_df = pd.concat([patient_med_df, patient_lab_df, patient_adm_df], axis=0, ignore_index=True)

,patientunitstayid,unittype,concept_name,concept_value,concept_table
0,141168,Med-Surg ICU,METOPROLOL TARTRATE 25 MG PO TABS,1.0,med
1,141168,Med-Surg ICU,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/3ML IN SOLN,1.0,med
2,141168,Med-Surg ICU,ASPIRIN EC 81 MG PO TBEC,1.0,med
3,141168,Med-Surg ICU,3 ML - IPRATROPIUM-ALBUTEROL 0.5-2.5 (3) MG/3ML IN SOLN,1.0,med
4,141168,Med-Surg ICU,ENOXAPARIN SODIUM 40 MG/0.4ML SC SOLN,1.0,med
...,...,...,...,...,...
47309299,3353117,SICU,ASPIRIN,1.0,adm
47309300,3353117,SICU,ATENOLOL,1.0,adm
47309301,3353117,SICU,SIMVASTATIN,1.0,adm
47309302,3353117,SICU,FENOFIBRATE,1.0,adm


In [ ]:
def filter_with_missing_rate(df, threshold):
    def filter_with_missing_rate_concept(x, nTotalPatients, threshold):
        nConceptPatients = len(x.patientunitstayid.unique())
        fEpsilon = 1.0e-08 # devide by zero
        fMissingRate = nConceptPatients / (nTotalPatients + fEpsilon)
        if (fMissingRate < threshold) :
            return pd.DataFrame(columns=x.columns)
        print("{}, {}, {}, {:.2}", list(x.concept_name)[0], nConceptPatients, nTotalPatients, fMissingRate)
        
        temp_df = pd.DataFrame(columns = ['concept_name', 'nConceptPatients', 'nTotalPatients', 'fMissingRate'])
        var_temp = {}
        var_temp['concept_name'] = list(x.concept_name)[0]
        var_temp['nConceptPatients'] = nConceptPatients
        var_temp['nTotalPatients'] = nTotalPatients
        var_temp['fMissingRate'] = fMissingRate
        temp_df = temp_df.append(var_temp, ignore_index=True)
        return temp_df

    nTotalPatients = len(df.patientunitstayid.unique())
    features_df = df.groupby('concept_name').apply(lambda x : filter_with_missing_rate_concept(x, nTotalPatients, threshold)).reset_index(drop=True)
    return features_df

concat_df.groupby('unittype').apply(lambda x : filter_with_missing_rate(x, nTotalPatients, threshold)).reset_index(drop=True)

df.loc[~df['concept_name'].isin(list(features_df.concept_name)), 'concept_name'] = None
df['value'] = 1
df = pd.pivot_table(data=df, index=['patientunitstayid', 'unittype'], columns='concept_name', values='value').reset_index()
df

In [42]:
pat_feature_list = []
apc_feature_list = []
med_feature_list = []
lab_feature_list = []
adm_feature_list = []
for icu in ['Med-Surg ICU','MICU','SICU']:
    pat_feature_list.append(set(patient_df.columns))
    apc_feature_list.append(set(apachepsvar_df.columns))
    med_feature_list.append(set(concat_df.loc[(concat_df['unittype']==icu) & (concat_df['concept_table']=='med')].concept_name.unique()))
    lab_feature_list.append(set(concat_df.loc[(concat_df['unittype']==icu) & (concat_df['concept_table']=='lab')].concept_name.unique()))
    adm_feature_list.append(set(concat_df.loc[(concat_df['unittype']==icu) & (concat_df['concept_table']=='adm')].concept_name.unique()))

In [43]:

for i in range(3):
    print(len(pat_feature_list[i]), len(apc_feature_list[i]), len(med_feature_list[i]), len(lab_feature_list[i]), len(adm_feature_list[i]))
    print(len(pat_feature_list[i])+ len(apc_feature_list[i])+ len(med_feature_list[i])+ len(lab_feature_list[i])+ len(adm_feature_list[i]))

29 26 1327 158 3759
5299
29 26 1129 154 1660
2998
29 26 981 151 1683
2870


In [30]:
total_set = set(pat_feature_list[1])|set(apc_feature_list[1])|set(med_feature_list[1])|set(lab_feature_list[1])|set(adm_feature_list[1])

import pickle
with open('micu_set.pickle', 'wb') as f:
    pickle.dump(total_set, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
med_features = set(patient_med_df['unitype'])

In [ ]:
medication_df['drughiclseqno'].apply(lambda x : int(x*1000/10000))

In [ ]:
len(medication_df.drughiclseqno.unique()), len(medication_df.drugname.unique())

In [ ]:
patient_med_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], medication_df[['patientunitstayid', 'drugname']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_med_df

In [ ]:
def filter_with_missing_rate_concept(x, nPatientInTotal, threshold):
    nPatientInConceptId = len(x.patientunitstayid.unique())
    fEpsilon = 1.0e-08 # devide by zero
    fMissingRate = nPatientInConceptId / (nPatientInTotal + fEpsilon)
    if (fMissingRate < threshold) :
        return pd.DataFrame(columns=x.columns)
    print("{}, {}, {}, {:.2}", list(x.drugname)[0], nPatientInConceptId, nPatientInTotal, fMissingRate)
    
    temp_df = pd.DataFrame(columns = ['drugname', 'nPatientInConceptId', 'nPatientInTotal', 'fMissingRate'])
    var_temp = {}
    var_temp['drugname'] = list(x.drugname)[0]
    var_temp['nPatientInConceptId'] = nPatientInConceptId
    var_temp['nPatientInTotal'] = nPatientInTotal
    var_temp['fMissingRate'] = fMissingRate
    temp_df = temp_df.append(var_temp, ignore_index=True)
    return temp_df

concept_list = []
nPatientInTotal = len(patient_df.patientunitstayid.unique())
drugs_df = patient_med_df.groupby('drugname').apply(lambda x : filter_with_missing_rate_concept(x, nPatientInTotal, threshold=0.05)).reset_index(drop=True).head(30)
patient_med_df.loc[~patient_med_df['drugname'].isin(list(drugs_df.drugname)), 'drugname'] = None
patient_med_df['value'] = 1
patient_med_df = pd.pivot_table(data=patient_med_df, index=['patientunitstayid', 'unittype'], columns='drugname', values='value').reset_index()
patient_med_df

In [ ]:
def resumetable(df):
    df = df.copy().replace(-1, np.NaN)
    print(f'data frame shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['data_type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'feature'})
    summary['n_missingvalues'] = df.isnull().sum().values
    summary['n_missingrates'] = 1 - df.isnull().sum().values/len(df)
    summary['n_eigenvalues'] = df.nunique().values
    return summary
patient_med_df = patient_med_df[patient_med_df['unittype'].isin(['Med-Surg ICU','MICU','SICU'])]
patient_med_summary_df = patient_med_df.groupby('unittype').apply(lambda x : resumetable(x)).reset_index()
patient_med_summary_df
heatmap_data = patient_med_summary_df[['unittype', 'drugname', 'n_missingrates']].pivot(index='drugname', columns='unittype', values='n_missingrates')
len(heatmap_data)
import seaborn as sns
sns.set(rc = {'figure.figsize':(8,16)})
sns.heatmap(data=heatmap_data, annot=True)

In [ ]:
patient_lab_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], lab_df[['patientunitstayid', 'labname', 'labresult']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_lab_df

def filter_with_missing_rate_concept(x, nPatientInTotal, threshold):
    nPatientInConceptId = len(x.patientunitstayid.unique())
    fEpsilon = 1.0e-08 # devide by zero
    fMissingRate = nPatientInConceptId / (nPatientInTotal + fEpsilon)
    if (fMissingRate < threshold) :
        return pd.DataFrame(columns=x.columns)
    print("{}, {}, {}, {:.2}", list(x.labname)[0], nPatientInConceptId, nPatientInTotal, fMissingRate)
    
    temp_df = pd.DataFrame(columns = ['labname', 'nPatientInConceptId', 'nPatientInTotal', 'fMissingRate'])
    var_temp = {}
    var_temp['labname'] = list(x.labname)[0]
    var_temp['nPatientInConceptId'] = nPatientInConceptId
    var_temp['nPatientInTotal'] = nPatientInTotal
    var_temp['fMissingRate'] = fMissingRate
    temp_df = temp_df.append(var_temp, ignore_index=True)
    return temp_df

concept_list = []
nPatientInTotal = len(patient_df.patientunitstayid.unique())
labs_df = patient_lab_df.groupby('labname').apply(lambda x : filter_with_missing_rate_concept(x, nPatientInTotal, threshold=0.5)).reset_index(drop=True).head(30)
patient_lab_df.loc[~patient_lab_df['labname'].isin(list(labs_df.labname)), 'labname'] = None
# patient_lab_df['labresult'] = 1
patient_lab_df = pd.pivot_table(data=patient_lab_df, index=['patientunitstayid', 'unittype'], columns='labname', values='labresult').reset_index()
patient_lab_df

In [ ]:
patient_lab_summary_df

In [ ]:
def resumetable(df):
    df = df.copy().replace(-1, np.NaN)
    print(f'data frame shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['data_type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'feature'})
    summary['n_missingvalues'] = df.isnull().sum().values
    summary['n_missingrates'] = 1 - df.isnull().sum().values/len(df)
    summary['n_eigenvalues'] = df.nunique().values
    return summary
patient_lab_df = patient_lab_df[patient_lab_df['unittype'].isin(['Med-Surg ICU','MICU','SICU'])]
patient_lab_summary_df = patient_lab_df.groupby('unittype').apply(lambda x : resumetable(x)).reset_index()
patient_lab_summary_df
heatmap_data = patient_lab_summary_df.pivot(index='labname', columns='unittype', values='n_missingrates')
len(heatmap_data)
import seaborn as sns
sns.set(rc = {'figure.figsize':(8,16)})
sns.heatmap(data=heatmap_data, annot=True)

In [ ]:
def resumetable(df):
    df = df.copy().replace(-1, np.NaN)
    print(f'data frame shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['data_type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'feature'})
    summary['n_missingvalues'] = df.isnull().sum().values
    summary['n_missingrates'] = df.isnull().sum().values/len(df)
    summary['n_eigenvalues'] = df.nunique().values
    return summary

summary = resumetable(patient_apc_df)
summary

def plot_missing_data_visualization(df, cols):
    import numpy as np
    import missingno as msno
    df = df.copy().replace(-1, np.NaN)  # -1 > np.NaN
    nCols = len(cols)
    msno.bar(df=df.loc[:,cols], figsize=(30, 20))
    # msno.matrix(df=df.loc[:,cols], figsize=(30, 6))
    
plot_missing_data_visualization(patient_apc_df, patient_apc_df.columns)

In [ ]:
summary['n_missingrates']

In [ ]:
patient_apc_df = patient_apc_df[patient_apc_df['unittype'].isin(['Med-Surg ICU','MICU','SICU'])]

In [ ]:
patient_apc_summary_df = patient_apc_df.groupby('unittype').apply(lambda x : resumetable(x)).reset_index()
patient_apc_summary_df

In [ ]:
heatmap_data = patient_apc_summary_df[['unittype', 'feature', 'n_missingrates']].pivot(index='feature', columns='unittype', values='n_missingrates')
len(heatmap_data)

In [ ]:
import seaborn as sns
sns.set(rc = {'figure.figsize':(8,16)})
sns.heatmap(data=heatmap_data, annot=True)

In [ ]:
patient_apc_df['unittype'].unique()

MSICU_patient_df = patient_apc_df[patient_apc_df['unittype']=='Med-Surg ICU']
MICU_patient_df = patient_apc_df[patient_apc_df['unittype']=='MICU']
SICU_patient_df = patient_apc_df[patient_apc_df['unittype']=='SICU']


In [ ]:
def resumetable(df):
    print(f'data frame shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['data_type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'feature'})
    summary['n_missingvalues'] = df.isnull().sum().values
    print(len(df))
    summary['n_missingrates'] = df.isnull().sum().values/len(df)*100
    summary['n_eigenvalues'] = df.nunique().values
    return summary

In [ ]:
summary = resumetable(patient_apc_df)
summary
plt.plot(summary.n_missingrates)


In [ ]:
MSICU_patient_df

In [ ]:
sql_query = '''
SELECT P.patientunitstayid, patienthealthsystemstayid, hospitaldischargestatus, gender, age, admissionheight, admissionweight, heartrate, meanbp, temperature, respiratoryrate
FROM eicu.PATIENT P
LEFT JOIN eicu.APACHEAPSVAR A
ON P.patientunitstayid = A.patientunitstayid
WHERE P.unittype = 'MICU'
'''
patient_df = pd.read_sql(sql_query, con=conn)

In [ ]:
patient_df